Goals

be able to access all data in S3 for a given season
- events table
- player and team match stats
- lineups and missing players table
- odds table


our end goal is to have player and team tables for the season which will facilitate making our features very easily. 
that means we should have a player table of every performance in the league with vaep, xG, rest days (have to incorporate european fixtures), travel distance (have to manually get coordinates for stadiums),  

then wrangle event data to get 

In [68]:
import boto3
from dotenv import load_dotenv
import os
import warnings
from io import StringIO
import pandas as pd
import socceraction.spadl as spadl

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
warnings.filterwarnings('ignore')

load_dotenv()
aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_access = os.getenv('AWS_SECRET_ACCESS')
aws_region = os.getenv('AWS_REGION')

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_access,
                region_name=aws_region)

bucket = 'footballbets'
league = "FRA-Ligue 1"
season = 2223

In [2]:
s3_event = s3.get_object(Bucket=bucket, Key=f'{league}/{season}/events_spadl.csv')
event_data = pd.read_csv(StringIO(s3_event['Body'].read().decode('utf-8')))

In [50]:
event_data.head()

,league,season,game,id,game_id,period,minute,second,expanded_minute,type,outcome_type,team_id,team,player_id,player,x,y,end_x,end_y,goal_mouth_y,goal_mouth_z,blocked_x,blocked_y,qualifiers,is_touch,is_shot,is_goal,card_type,related_event_id,related_player_id
0,FRA-Ligue 1,2223,2022-08-06 Strasbourg-Monaco,2442542229,1643708,PreMatch,0,0.0,0,FormationSet,Successful,248,Monaco,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'type': {'displayName': 'PlayerPosition', 'v...",False,NaN,NaN,NaN,NaN,NaN
1,FRA-Ligue 1,2223,2022-08-06 Strasbourg-Monaco,2442546829,1643708,PreMatch,0,0.0,0,FormationSet,Successful,148,Strasbourg,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'type': {'displayName': 'InvolvedPlayers', '...",False,NaN,NaN,NaN,NaN,NaN
2,FRA-Ligue 1,2223,2022-08-06 Strasbourg-Monaco,2442594203,1643708,FirstHalf,0,0.0,0,Start,Successful,148,Strasbourg,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,[],False,NaN,NaN,NaN,NaN,NaN
3,FRA-Ligue 1,2223,2022-08-06 Strasbourg-Monaco,2442594217,1643708,FirstHalf,0,0.0,0,Start,Successful,248,Monaco,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,[],False,NaN,NaN,NaN,NaN,NaN
4,FRA-Ligue 1,2223,2022-08-06 Strasbourg-Monaco,2442594285,1643708,FirstHalf,0,0.0,0,Pass,Successful,148,Strasbourg,113747.0,Adrien Thomasson,50.0,50.0,32.9,37.6,NaN,NaN,NaN,NaN,"[{'type': {'displayName': 'PassEndY', 'value':...",True,NaN,NaN,NaN,NaN,NaN


In [49]:
event_data.type.unique()

array(['FormationSet', 'Start', 'Pass', 'Aerial', 'BallRecovery',
       'OffsidePass', 'OffsideProvoked', 'BallTouch', 'Interception',
       'KeeperPickup', 'CornerAwarded', 'Clearance', 'Dispossessed',
       'Tackle', 'SavedShot', 'Save', 'BlockedPass', 'Foul', 'TakeOn',
       'OffsideGiven', 'Challenge', 'MissedShots', 'Claim', 'Goal', 'End',
       'SubstitutionOff', 'SubstitutionOn', 'KeeperSweeper', 'Card',
       'FormationChange', 'Error', 'PenaltyFaced', 'GoodSkill', 'Punch',
       'Smother', 'ShotOnPost', 'ShieldBallOpp'], dtype=object)

In [ ]:
[item for item in event_data[event_data['type'] == 'Pass'].qualifiers]

In [ ]:
next_event = event_data.shift(-1, fill_value=0)


In [42]:
spadl_e = s3.get_object(Bucket=bucket, Key=f'ENG-Premier League/2223/events_spadl.csv')
spadldf = pd.read_csv(StringIO(spadl_e['Body'].read().decode('utf-8')))
spadldf = spadl.add_names(spadldf)

In [213]:
sample[(sample['nextEvent'].isin(['interception', 'tackle'])) & (sample['result_name'] == 'success')].type_name.value_counts()

type_name
pass           11
dribble         8
clearance       1
keeper_save     1
Name: count, dtype: int64

In [93]:
sample.result_name.value_counts()

result_name
success    1339
fail        346
Name: count, dtype: int64

In [58]:
sample = spadldf[spadldf['game_id'] == 1640952]

In [236]:
next_event = sample.shift(-1, fill_value=0)
sample["nextEvent"] = next_event["type_name"]
sample['kickedOut'] = sample.apply(lambda x: 1 if x['nextEvent'] == 'throw_in' else 0, axis=1)

sample['nextTeamId'] = next_event['team_id']
chain_team = sample.iloc[0]['team_id']
period = sample.iloc[0]['period_id']

stop_criterion = 0
chain = 0
sample['possession_chain'] = 0
sample['possession_chain_team'] = 0

for i, row in sample.iterrows():
    sample.at[i, 'possession_chain'] = chain
    sample.at[i, 'possession_chain_team'] = chain_team

    if row.type_name in ['pass', 'duel', 'dribble']:

        if row['result_name'] == 'fail':
            if row.nextEvent == 'interception' or row.nextEvent == 'tackle':
                stop_criterion += 2
            else:
                stop_criterion +=1

        if row.team_id != row.nextTeamId:
            if row.nextEvent == 'dribble' or row.nextEvent == 'pass' or row.nextEvent == 'tackle':
                stop_criterion += 2

    # if row.type_name == 'interception' or row.type_name == 'tackle':
    #     if row.result_name == 'success':
    #         stop_criterion += 2

    if row.type_name in ['pass', 'cross', 'freekick_crossed', 'corner_crossed']:
        if row.result_name == 'offside':
            stop_criterion += 2
    if row.type_name in ['shot', 'foul', 'clearance']:
        stop_criterion += 2
    if row['kickedOut'] == 1:
        stop_criterion += 2

    if row['period_id'] != period:
        chain += 1
        stop_criterion = 0
        chain_team = row['team_id']
        period = row['period_id']
        sample.at[i, 'possession_chain'] = chain
        sample.at[i, 'possession_chain_team'] = chain_team
    
    if stop_criterion >= 2:
        chain += 1
        stop_criterion = 0
        chain_team = row['nextTeamId']


In [261]:
sample[sample['possession_chain'] == 19]

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,end_x,start_y,end_y,type_id,result_id,bodypart_id,action_id,player,team,type_name,result_name,bodypart_name,nextEvent,kickedOut,nextTeamId,possession_chain,possession_chain_team
80,1640952,2.510007e+09,1,329.0,30,101859.0,75.495,82.215,46.784,58.888,0,1,1,80,Pierre-Emile Højbjerg,Tottenham,pass,success,head,pass,0,30,19,30
81,1640952,2.510007e+09,1,331.0,30,103837.0,83.055,95.760,59.228,57.868,0,1,0,81,Ben Davies,Tottenham,pass,success,foot,dribble,0,30,19,30
82,1640952,NaN,1,332.5,30,317804.0,95.760,96.075,57.868,61.336,21,1,0,82,Richarlison,Tottenham,dribble,success,foot,pass,0,30,19,30
83,1640952,2.510007e+09,1,334.0,30,317804.0,96.075,83.370,61.336,64.668,0,1,0,83,Richarlison,Tottenham,pass,success,foot,pass,0,30,19,30
84,1640952,2.510007e+09,1,337.0,30,103837.0,81.585,72.030,63.240,55.216,0,1,0,84,Ben Davies,Tottenham,pass,success,foot,dribble,0,30,19,30
85,1640952,NaN,1,338.5,30,101859.0,72.030,66.570,55.216,49.844,21,1,0,85,Pierre-Emile Højbjerg,Tottenham,dribble,success,foot,pass,0,30,19,30
86,1640952,2.510007e+09,1,340.0,30,101859.0,66.570,66.675,49.844,17.544,0,1,0,86,Pierre-Emile Højbjerg,Tottenham,pass,success,foot,pass,0,30,19,30
87,1640952,2.510008e+09,1,344.0,30,323663.0,67.830,73.290,14.892,4.556,0,1,0,87,Cristian Romero,Tottenham,pass,success,foot,pass,0,30,19,30
88,1640952,2.510008e+09,1,345.0,30,355855.0,74.655,74.970,4.556,12.580,0,1,0,88,Dejan Kulusevski,Tottenham,pass,success,foot,dribble,0,30,19,30
89,1640952,NaN,1,346.5,30,328512.0,74.970,73.815,12.580,20.060,21,1,0,89,Emerson Royal,Tottenham,dribble,success,foot,pass,0,30,19,30


In [33]:
import soccerdata as sd
fbref = sd.FBref(leagues=league, seasons=season)

[07/29/24 09:27:18] INFO     Saving cached data to C:\Users\rsacc\soccerdata\data\FBref              ]8;id=88852;file://c:\Users\rsacc\Desktop\footballbets\venv\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=941327;file://c:\Users\rsacc\Desktop\footballbets\venv\Lib\site-packages\soccerdata\_common.py#263\263]8;;\

In [34]:
schedule = fbref.read_schedule()

In [35]:
schedule.iloc[:380].index[0]

('ENG-Premier League', '2223', '2022-08-05 Crystal Palace-Arsenal')